In [11]:
# Install Sckit-learn Optimization Library
!pip install scikit-optimize

     |████████████████████████████████| 102kB 2.3MB/s 


In [16]:
import pandas as pd
import numpy as np
import warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import datasets
from functools import partial
from skopt import space,gp_minimize

warnings.filterwarnings("ignore")

In [5]:
 #--------- Custom Dataset -----------#

iris = datasets.load_iris()
X = iris.data[:,:2]
y = iris.target

 #--------- Custom Dataset -----------#

## Bayesian Optimization using Gaussian Process

In [8]:
def optimize(params,param_names,x,y):
  params = dict(zip(param_names,params))

  #--------- Custom Code/Model -----------#

  model = RandomForestClassifier(**params)
  kf = StratifiedKFold(n_splits=5)

  #--------- Custom Code/Model -----------#

  # Cross Validation split
  accuracies = []
  for idx in kf.split(X=x,y=y):
    train_idx,val_idx = idx[0],idx[1]
    # Train
    Xtrain = x[train_idx]
    ytrain = y[train_idx]
    # Validation
    Xval = x[val_idx]
    yval = y[val_idx]

    model.fit(Xtrain,ytrain)
    pred = model.predict(Xval)
    fold_acc = accuracy_score(yval,pred)
    accuracies.append(fold_acc)
  
  return -1.0 * np.mean(accuracies)

In [15]:
param_space = [
    space.Integer(3,15,name="max_depth"),
    space.Integer(100,600,name="n_estimators"),
    space.Categorical(["gini","entropy"],name="criterion"),
    space.Real(0.01,1,prior="uniform",name="max_features")
]

param_names = [
    "max_depth",
    "n_estimators",
    "criterion",
    "max_features"
]

optimization_function = partial(optimize,param_names=param_names,x = X,y=y)

In [18]:
result = gp_minimize(optimization_function,
                     dimensions=param_space,
                     n_calls=15,
                     n_random_starts=10,
                     verbose=10)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.8340
Function value obtained: -0.8067
Current minimum: -0.8067
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.7087
Function value obtained: -0.7400
Current minimum: -0.8067
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 2.8642
Function value obtained: -0.7467
Current minimum: -0.8067
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.8689
Function value obtained: -0.7200
Current minimum: -0.8067
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.4288
Function value obtained: -0.7400
Current minimum: -0.8067
Iteration No: 6 started. 

In [20]:
print(dict(zip(param_names,result.x)))

{'max_depth': 4, 'n_estimators': 429, 'criterion': 'entropy', 'max_features': 0.7333104175258193}
